In [11]:
from tensorflow.keras.datasets.mnist import load_data
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import time
from tqdm import tqdm
%matplotlib inline

(X_train, Y_train), (X_test, Y_test) = load_data()
print("X_train: ", len(X_train), "件")
print("Y_train: ", len(Y_train), "件")
print("X_test : ", len(X_test), "件")
print("Y_test : ", len(Y_test), "件")

X_train:  60000 件
Y_train:  60000 件
X_test :  10000 件
Y_test :  10000 件


In [12]:
# 読み込み直後のデータ形式
print("X_train.shape: ", X_train.shape)
print("X_test.shape : ", X_test.shape)
print("Y_train.shape: ", Y_train.shape)
print("Y_test.shape : ", Y_test.shape)

X_train.shape:  (60000, 28, 28)
X_test.shape :  (10000, 28, 28)
Y_train.shape:  (60000,)
Y_test.shape :  (10000,)


In [13]:
# 画像データの変換
X_train = X_train.reshape(60000, 28 * 28)
X_test = X_test.reshape(10000, 28 * 28)
print("変換後X_train: ", X_train.shape)
print("変換後X_test : ", X_test.shape)

変換後X_train:  (60000, 784)
変換後X_test :  (10000, 784)


In [14]:
# parameters-------------------------------------
k_list = [3, 5, 7, 9, 11, 13, 15] # knnのkの値を設定
test_size = 0.01 # 検証データ
random_state = 60 # train_test_split()のrandom_stateの引数
# -----------------------------------------------

# 全60,000件の学習データのうち，test_sizeで指定した割合のデータを検証用データにとっておく
trData, valData, trLabels, valLabels = train_test_split(np.array(X_train), Y_train, test_size=test_size, random_state=random_state)

# 何%を検証用にするか
nVals = np.arange(0.9, 0.0, -0.1)

accuracies = []
times = []
pridicts = []

for k in k_list:
    accuracy = []  # 正確度保存用
    proc_time = []   # 実行時間保存用
    for per in tqdm( nVals ):  # tqdmを使うとき
        # 48000件のデータのうち，per％をテスト用に，残りを学習用として確保
        trainData, testData, trainLabels, testLabels = train_test_split(trData, trLabels, test_size=per, random_state=42)
        start = time.time()  # 時間計測開始

        # NNモデル
        model = KNeighborsClassifier(n_neighbors=k)
        model.fit(trainData, trainLabels)

        # 検証用データを使って実行
        score = model.score(valData, valLabels)    

        duration = time.time() - start  # 計測終了

        print("train size=%d, accuracy=%.2f%%, time=%.2f[s]" % (len(trainLabels), score * 100, duration))
        # 正確度と処理時間を保存
        accuracy.append(score)
        proc_time.append(duration)
        
        # モデルの予想
        val_predict = model.predict(valData)
        
        wrong_list = []
        wrong_value = []
        correct_value = []
        for i in range(len(val_predict)):
            if val_predict[i] != valLabels[i]:
                wrong_list.append(i)
                wrong_value.append(val_predict[i])
                correct_value.append(valLabels[i])
        plt.figure(figsize=(14,10))
        for i in range(len(correct_value)):
            ax = plt.subplot(3, 6, i + 1)
            ax.set_axis_off()
            ax.set_title( "correct: {}, wrong:{}".format(correct_value[i], wrong_value[i]))
            plt.imshow(valData[wrong_list[i]].reshape(28, 28).astype("uint8"))
        plt.show()

        # 生確度と処理時間の結果を別ファイルに保存
        resultfilename = "k=" + str(k) + "_train" + str((1-test_size)*100) + "%_accuracies.txt"
        with open(resultfilename, mode='w') as f:
            for d in accuracies:
                f.write("%s\n" % d)

        resultfilename = "k=" + str(k) + "_train" + str((1-test_size)*100) + "%_times.txt"
        with open(resultfilename, mode='w') as f:
            for d in proc_time:
                f.write("%s\n" % d)
        
    accuracies.append(accuracy)
    times.append(proc_time)

 11%|█████████▎                                                                          | 1/9 [00:02<00:23,  2.89s/it]

train size=3000, accuracy=91.83%, time=2.87[s]
train size= 2999 achieved highest accuracy of 91.83% on validation data


 22%|██████████████████▋                                                                 | 2/9 [00:08<00:29,  4.25s/it]

train size=6000, accuracy=93.68%, time=5.19[s]
train size= 5999 achieved highest accuracy of 93.68% on validation data


 33%|████████████████████████████                                                        | 3/9 [00:15<00:34,  5.79s/it]

...

train size=20999, accuracy=94.92%, time=22.43[s]
train size=20999 achieved highest accuracy of 94.92% on validation data


 89%|██████████████████████████████████████████████████████████████████████████▋         | 8/9 [02:06<00:20, 20.72s/it]

train size=23999, accuracy=95.17%, time=25.50[s]
train size=23999 achieved highest accuracy of 95.17% on validation data


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:35<00:00, 17.32s/it]

train size=26999, accuracy=95.40%, time=28.93[s]
train size=26999 achieved highest accuracy of 95.40% on validation data


In [ ]:
plt.clf()
plt.rcParams['font.size'] = 16

# グラフ用
fig, ax1 = plt.subplots(figsize=(12,10))

# x軸は学習データ数にする
x = len(trLabels) * (1-nVals)
plt.xlabel('Number of training samples')

# 1軸に識別率
ax1.plot(x, np.array(accuracies), color = 'r', label = 'accuracies')
# 1軸と2軸の関連づけ
plt.ylabel('Accuracies')
plt.legend()
ax2 = ax1.twinx()
# 2軸に処理時間
ax2.plot(x, np.array(proc_time), 'b', label = 'processing time')
plt.ylabel('Processing Time [s]')

# ファイル保存したいとき
name = 'nn_k=' + str(k) + '_compare.png'
plt.savefig(name)

plt.show()